In [ ]:
## smooth height as before:
hght_700 = mpcalc.smooth_n_point(hght_700, 5)

### smooth everything else pretty heavily:
temp_700 = mpcalc.smooth_gaussian(temp_700, 12) 
uwnd_700 = mpcalc.smooth_gaussian(uwnd_700, 12) 
vwnd_700 = mpcalc.smooth_gaussian(vwnd_700, 12) 

In [ ]:
# Compute the Q-vector components
uqvect, vqvect = mpcalc.q_vector(uwnd_700, vwnd_700, temp_700, 700*units.hPa)

# Compute the divergence of the Q-vectors calculated above
q_div = -2*mpcalc.divergence(uqvect, vqvect)

## smooth the divergence too
q_div = mpcalc.smooth_gaussian(q_div, 18) 

In [ ]:
q_div = ndimage.gaussian_filter(q_div, sigma=10, order=0) * units('meter/(kilogram second)')
q_div.units

In [ ]:
#Map Creation

# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=45, central_longitude=-100)

fig = plt.figure(1, figsize=(20,16))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, .02], bottom=.07, top=.99,
                       hspace=0.01, wspace=0.01)
ax = plt.subplot(gs[0], projection=plotcrs)

# Plot Titles
plt.title('GFS analysis 700-hPa Geo. Heights (m), Temp (C),'
          ' Q-Vectors (m$^2$ kg$^{-1}$ s$^{-1}$) and Q-Vec Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)', 
          loc='left', fontsize=16)
plt.title(vtime.strftime("%H%M UTC %d %b %Y"), loc='right', fontsize=18)

# Plot Background
#ax.set_extent([217.5, 290., 18., 66.])
ax.set_extent([227.5, 290., 20., 56.])
ax.coastlines('50m', edgecolor='black', linewidth=0.75)
ax.add_feature(cfeature.STATES, linewidth=.5)

# Plot Height Contours
clev700 = np.arange(1800, 4500, 30)
cs = ax.contour(lon, lat, hght_700, clev700, colors='black', linewidths=1.0,
                linestyles='solid', transform=ccrs.PlateCarree())
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot temperature Contours
clevtemp700 = np.arange(-40, 20, 2)
cs2 = ax.contour(lon, lat, temp_700.metpy.convert_units('degC'), clevtemp700, colors='grey',
                 linewidths=1.25, linestyles='dashed', transform=ccrs.PlateCarree())
plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot 850-hPa Q-Vector Divergence and scale
clevs_qdiv = list(range(-30, -4, 5))+list(range(5, 31, 5))
cf = ax.contourf(lon, lat, q_div*1e18, clevs_qdiv, extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = plt.colorbar(cf, orientation='horizontal', ticks=clevs_qdiv, shrink=0.9, pad=0.02, aspect=35)
cb.set_label('Q-Vector Div. (*10$^{18}$ m s$^{-1}$ kg$^{-1}$)')

# Plot 700-hPa Q-vectors, scale to get nice sized arrows. Skip every 5th point when plotting.
ax.quiver(lon[::5],lat[::5],uqvect[::5,::5].values,vqvect[::5,::5].values,
          pivot='mid', color='black',
          scale=1e-11, scale_units='inches',
          transform=ccrs.PlateCarree())

### uncomment these if you'd like to save a file
#outfile="700qvect_gfs_"+vtime.strftime('%Y%m%d')+"_"+vtime.strftime('%H%M')+".png"
#fig.savefig(outfile, dpi=255, bbox_inches='tight', transparent=False, facecolor='white')

plt.show()   ### show in the notebook

plt.close('all')